<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Testing-approach-on-one-language" data-toc-modified-id="Testing-approach-on-one-language-1">Testing approach on one language</a></span></li><li><span><a href="#Batch-processing-for-all-languages" data-toc-modified-id="Batch-processing-for-all-languages-2">Batch processing for all languages</a></span><ul class="toc-item"><li><span><a href="#Sort-by-percent-of-filled-refs" data-toc-modified-id="Sort-by-percent-of-filled-refs-2.1">Sort by percent of filled refs</a></span></li><li><span><a href="#Sort-by-percent-with-threshold" data-toc-modified-id="Sort-by-percent-with-threshold-2.2">Sort by percent with threshold</a></span></li><li><span><a href="#Sort-by-percent-filled-in-when-all-languages-are-considered" data-toc-modified-id="Sort-by-percent-filled-in-when-all-languages-are-considered-2.3">Sort by percent filled in when all languages are considered</a></span></li><li><span><a href="#Export-to-LaTeX" data-toc-modified-id="Export-to-LaTeX-2.4">Export to LaTeX</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML, display_html 
import seaborn as sns
import networkx as nx

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 20)
sns.set(rc={'figure.figsize':(15,10)})
sns.set_style("whitegrid")

#%matplotlib inline

print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Matplotlib: {mpl.__version__}")
print(f"NetworkX: {nx.__version__}")

Pandas: 1.1.4
NumPy: 1.19.3
Matplotlib: 3.3.2
NetworkX: 2.4


# Testing approach on one language

In [11]:
LANG = "de"
FILE = f"stats1u/{LANG}_infobox-mapped-properties-withref.txt"
df_stats1u = pd.read_csv(FILE, sep="\t", names=['uri', 'cnt_refs'])

In [12]:
df_stats1u

,uri,cnt_refs
0,http://dbpedia.org/ontology/numberOfEmployees,7411
1,http://purl.org/dc/elements/1.1/description,6861
2,http://dbpedia.org/ontology/meltingPoint,6694
3,http://dbpedia.org/ontology/ChemicalSubstance/...,6694
4,http://dbpedia.org/ontology/revenue,5929
...,...,...
393,http://dbpedia.org/ontology/demolitionDate,1
394,http://dbpedia.org/ontology/dateOfAbandonment,1
395,http://dbpedia.org/ontology/notableFeatures,1
396,http://dbpedia.org/ontology/cluster,1


In [13]:
LANG = "de"
FILE = f"stats3/{LANG}_properties_mappingbased-literals.txt"
df_stats3 = pd.read_csv(FILE, sep="\t", names=['uri', 'cnt_total'])

In [14]:
df_stats3

,uri,cnt_total
0,http://xmlns.com/foaf/0.1/name,1308646
1,http://dbpedia.org/ontology/birthDate,643915
2,http://dbpedia.org/ontology/viafId,548404
3,http://dbpedia.org/ontology/individualisedGnd,537057
4,http://xmlns.com/foaf/0.1/nick,357273
...,...,...
305,http://dbpedia.org/ontology/floorArea,5
306,http://dbpedia.org/ontology/lunarOrbitTime,3
307,http://dbpedia.org/ontology/completionDate,3
308,http://dbpedia.org/ontology/cmpEvaDuration,1


In [19]:
dfx = df_stats1u.merge(df_stats3, on='uri')
dfx['filled_pcnt'] = dfx.cnt_refs/dfx.cnt_total
dfx.sort_values('filled_pcnt', ascending=False).head(20)

,uri,cnt_refs,cnt_total,filled_pcnt
55,http://dbpedia.org/ontology/density,103,103,1.000000
76,http://dbpedia.org/ontology/sublimationPoint,56,57,0.982456
6,http://dbpedia.org/ontology/boilingPoint,3379,3466,0.974899
2,http://dbpedia.org/ontology/meltingPoint,6694,6884,0.972400
136,http://dbpedia.org/ontology/areaOfCatchmentQuote,9,11,0.818182
159,http://dbpedia.org/ontology/volumeQuote,4,5,0.800000
110,http://dbpedia.org/ontology/areaQuote,23,30,0.766667
140,http://dbpedia.org/ontology/elevationQuote,8,11,0.727273
3,http://dbpedia.org/ontology/revenue,5929,9754,0.607853
167,http://dbpedia.org/ontology/averageDepthQuote,3,5,0.600000


# Batch processing for all languages

In [6]:
langs = ['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ru', 'sv', 'uk']
dataframes = dict()
for lang in langs:
    file = f"stats1u/{lang}_infobox-mapped-properties-withref.txt"
    df_stats1u = pd.read_csv(file, sep="\t", names=['uri', f'{lang}_refs'])
    file = f"stats3/{lang}_properties_mappingbased-literals.txt"
    df_stats3 = pd.read_csv(file, sep="\t", names=['uri', f'{lang}_total'])
    dfx = df_stats1u.merge(df_stats3, on='uri')
    dfx[f'{lang}_pcnt'] = dfx[f'{lang}_refs']/dfx[f'{lang}_total']
    dataframes[lang] = dfx
df_merged = dataframes['de'].copy()
for lang in langs[1:]:
    df_merged = df_merged.merge(dataframes[lang], 'outer', on='uri')
# df_merged.fillna(0, inplace=True)
cols_refs = [f'{l}_refs' for l in langs]
df_merged['num_langs'] = (df_merged[cols_refs]>0).astype(int).sum(axis=1)
df_merged['all_refs'] = df_merged[cols_refs].sum(axis=1).astype(int)
cols_tot = [f'{l}_total' for l in langs]
df_merged['all_total'] = df_merged[cols_tot].sum(axis=1).astype(int)
df_merged['all_pcnt'] = df_merged.all_refs/df_merged.all_total
df_merged.set_index('uri', inplace=True)
df_merged.sort_values(['num_langs', 'all_pcnt'], ascending=False)

,de_refs,de_total,de_pcnt,en_refs,en_total,en_pcnt,es_refs,es_total,es_pcnt,fr_refs,fr_total,fr_pcnt,it_refs,it_total,it_pcnt,nl_refs,nl_total,nl_pcnt,pl_refs,pl_total,pl_pcnt,pt_refs,pt_total,pt_pcnt,ru_refs,ru_total,ru_pcnt,sv_refs,sv_total,sv_pcnt,uk_refs,uk_total,uk_pcnt,num_langs,all_refs,all_total,all_pcnt
uri,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
http://dbpedia.org/ontology/numberOfStudents,2131.0,5997.0,0.355344,14992.0,33752.0,0.444181,384.0,2004.0,0.191617,390.0,2125.0,0.183529,60.0,1153.0,0.052038,162.0,1118.0,0.144902,348.0,1187.0,0.293176,89.0,336.0,0.264881,231.0,1369.0,0.168736,105.0,335.0,0.313433,8.0,154.0,0.051948,11,18900,49530,0.381587
http://dbpedia.org/ontology/foundingDate,42.0,3367.0,0.012474,4145.0,35410.0,0.117057,4.0,36.0,0.111111,45.0,3481.0,0.012927,25.0,1865.0,0.013405,16.0,271.0,0.059041,87.0,705.0,0.123404,3.0,58.0,0.051724,16.0,253.0,0.063241,10.0,294.0,0.034014,2.0,128.0,0.015625,11,4395,45868,0.095818
http://dbpedia.org/ontology/releaseDate,659.0,15119.0,0.043588,9490.0,130756.0,0.072578,39.0,207.0,0.188406,1609.0,43730.0,0.036794,1.0,16.0,0.062500,398.0,10839.0,0.036719,955.0,7982.0,0.119644,10.0,148.0,0.067568,653.0,12155.0,0.053723,8.0,68.0,0.117647,9.0,99.0,0.090909,11,13831,221119,0.062550
http://dbpedia.org/ontology/weight,102.0,21384.0,0.004770,6337.0,103397.0,0.061288,448.0,6394.0,0.070066,102.0,6063.0,0.016823,1317.0,19869.0,0.066284,170.0,8882.0,0.019140,202.0,2303.0,0.087712,197.0,11475.0,0.017168,2.0,244.0,0.008197,61.0,1304.0,0.046779,1.0,13.0,0.076923,11,8939,181328,0.049297
http://dbpedia.org/ontology/postalCode,2010.0,151889.0,0.013233,2756.0,211078.0,0.013057,8560.0,111240.0,0.076951,21234.0,96986.0,0.218939,4650.0,22393.0,0.207654,80.0,115204.0,0.000694,5114.0,113158.0,0.045193,305.0,64946.0,0.004696,6462.0,161582.0,0.039992,7.0,54029.0,0.000130,31.0,68559.0,0.000452,11,51209,1171064,0.043729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
http://dbpedia.org/ontology/draft,NaN,NaN,NaN,1.0,10457.0,0.000096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,10457,0.000096
http://dbpedia.org/ontology/oldName,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,283224.0,0.000074,NaN,NaN,NaN,1,21,283224,0.000074
http://dbpedia.org/ontology/routeStartDirection,NaN,NaN,NaN,1.0,19262.0,0.000052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,19262,0.000052


In [64]:
df_merged.loc['http://dbpedia.org/ontology/populationTotal','en_refs']

7025.0

## Sort by percent of filled refs

In [9]:
df_merged[df_merged.num_langs>0].sort_values(['all_pcnt'], ascending=False).head(20)

,de_refs,de_total,de_pcnt,en_refs,en_total,en_pcnt,es_refs,es_total,es_pcnt,fr_refs,fr_total,fr_pcnt,it_refs,it_total,it_pcnt,nl_refs,nl_total,nl_pcnt,pl_refs,pl_total,pl_pcnt,pt_refs,pt_total,pt_pcnt,ru_refs,ru_total,ru_pcnt,sv_refs,sv_total,sv_pcnt,uk_refs,uk_total,uk_pcnt,num_langs,all_refs,all_total,all_pcnt
uri,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
http://dbpedia.org/ontology/gdpPerCapita,NaN,NaN,NaN,2.0,2.0,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2,2,1.000000
http://dbpedia.org/ontology/nrhpReferenceNumber,NaN,NaN,NaN,59478.0,65696.0,0.905352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,59478,65696,0.905352
http://dbpedia.org/ontology/barPassRate,NaN,NaN,NaN,137.0,164.0,0.835366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,137,164,0.835366
http://dbpedia.org/ontology/areaOfCatchmentQuote,9.0,11.0,0.818182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,9,11,0.818182
http://dbpedia.org/ontology/volumeQuote,4.0,5.0,0.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4,5,0.800000
http://dbpedia.org/ontology/areaQuote,23.0,30.0,0.766667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,23,30,0.766667
http://dbpedia.org/ontology/nameDay,NaN,NaN,NaN,3.0,4.0,0.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3,4,0.750000
http://dbpedia.org/ontology/giniCoefficient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.0,91.0,0.736264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,67,91,0.736264
http://dbpedia.org/ontology/elevationQuote,8.0,11.0,0.727273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,8,11,0.727273


**Conclusions:** there are a lot of NaN values, i.e. attribute is not used and not references provided.

## Sort by percent with threshold

Let's add the requirements that at least 3 languages should use the property.

In [10]:
df_merged[df_merged.num_langs>=3].sort_values(['all_pcnt'], ascending=False).head(20)

,de_refs,de_total,de_pcnt,en_refs,en_total,en_pcnt,es_refs,es_total,es_pcnt,fr_refs,fr_total,fr_pcnt,it_refs,it_total,it_pcnt,nl_refs,nl_total,nl_pcnt,pl_refs,pl_total,pl_pcnt,pt_refs,pt_total,pt_pcnt,ru_refs,ru_total,ru_pcnt,sv_refs,sv_total,sv_pcnt,uk_refs,uk_total,uk_pcnt,num_langs,all_refs,all_total,all_pcnt
uri,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
http://dbpedia.org/ontology/gross,NaN,NaN,NaN,16682.0,21936.0,0.760485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2423.0,3836.0,0.631648,1792.0,3736.0,0.479657,NaN,NaN,NaN,27.0,57.0,0.473684,4,20924,29565,0.707729
http://dbpedia.org/ontology/density,103.0,103.0,1.000000,61.0,133.0,0.458647,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,165,238,0.693277
http://dbpedia.org/ontology/endowment,203.0,426.0,0.476526,1547.0,2034.0,0.760570,81.0,207.0,0.391304,38.0,95.0,0.400000,NaN,NaN,NaN,5.0,26.0,0.192308,NaN,NaN,NaN,49.0,77.0,0.636364,1.0,5.0,0.200000,NaN,NaN,NaN,NaN,NaN,NaN,7,1924,2870,0.670383
http://dbpedia.org/ontology/salary,NaN,NaN,NaN,1.0,1.0,1.000000,20.0,34.0,0.588235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,10.0,0.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,29,45,0.644444
http://dbpedia.org/ontology/meltingPoint,6694.0,6884.0,0.972400,NaN,NaN,NaN,NaN,NaN,NaN,15.0,54.0,0.277778,NaN,NaN,NaN,175.0,2746.0,0.063729,7.0,2131.0,0.003285,NaN,NaN,NaN,1.0,4.0,0.250000,NaN,NaN,NaN,NaN,NaN,NaN,5,6892,11819,0.583129
http://dbpedia.org/ontology/boilingPoint,3379.0,3466.0,0.974899,NaN,NaN,NaN,NaN,NaN,NaN,4.0,21.0,0.190476,79.0,897.0,0.088071,122.0,1841.0,0.066268,5.0,1022.0,0.004892,NaN,NaN,NaN,2.0,5.0,0.400000,NaN,NaN,NaN,NaN,NaN,NaN,6,3591,7252,0.495174
http://dbpedia.org/ontology/operatingIncome,NaN,NaN,NaN,2013.0,3449.0,0.583647,307.0,701.0,0.437946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,0.250000,19.0,583.0,0.032590,NaN,NaN,NaN,4,2340,4737,0.493984
http://dbpedia.org/ontology/totalPopulation,NaN,NaN,NaN,2055.0,3831.0,0.536413,227.0,789.0,0.287706,NaN,NaN,NaN,NaN,NaN,NaN,75.0,412.0,0.182039,NaN,NaN,NaN,307.0,662.0,0.463746,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,2664,5694,0.467861
http://dbpedia.org/ontology/netIncome,NaN,NaN,NaN,2688.0,5351.0,0.502336,265.0,628.0,0.421975,1237.0,2623.0,0.471597,370.0,951.0,0.389064,357.0,1006.0,0.354871,NaN,NaN,NaN,46.0,79.0,0.582278,3.0,5.0,0.600000,NaN,NaN,NaN,NaN,NaN,NaN,7,4966,10643,0.466598


## Sort by percent filled in when all languages are considered
We can also require that all 11 languages are present in calculations.

In [11]:
df_merged[df_merged.num_langs==11].sort_values(['all_pcnt'], ascending=False).head(20)

,de_refs,de_total,de_pcnt,en_refs,en_total,en_pcnt,es_refs,es_total,es_pcnt,fr_refs,fr_total,fr_pcnt,it_refs,it_total,it_pcnt,nl_refs,nl_total,nl_pcnt,pl_refs,pl_total,pl_pcnt,pt_refs,pt_total,pt_pcnt,ru_refs,ru_total,ru_pcnt,sv_refs,sv_total,sv_pcnt,uk_refs,uk_total,uk_pcnt,num_langs,all_refs,all_total,all_pcnt
uri,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
http://dbpedia.org/ontology/numberOfStudents,2131.0,5997.0,0.355344,14992.0,33752.0,0.444181,384.0,2004.0,0.191617,390.0,2125.0,0.183529,60.0,1153.0,0.052038,162.0,1118.0,0.144902,348.0,1187.0,0.293176,89.0,336.0,0.264881,231.0,1369.0,0.168736,105.0,335.0,0.313433,8.0,154.0,0.051948,11,18900,49530,0.381587
http://dbpedia.org/ontology/foundingDate,42.0,3367.0,0.012474,4145.0,35410.0,0.117057,4.0,36.0,0.111111,45.0,3481.0,0.012927,25.0,1865.0,0.013405,16.0,271.0,0.059041,87.0,705.0,0.123404,3.0,58.0,0.051724,16.0,253.0,0.063241,10.0,294.0,0.034014,2.0,128.0,0.015625,11,4395,45868,0.095818
http://dbpedia.org/ontology/releaseDate,659.0,15119.0,0.043588,9490.0,130756.0,0.072578,39.0,207.0,0.188406,1609.0,43730.0,0.036794,1.0,16.0,0.062500,398.0,10839.0,0.036719,955.0,7982.0,0.119644,10.0,148.0,0.067568,653.0,12155.0,0.053723,8.0,68.0,0.117647,9.0,99.0,0.090909,11,13831,221119,0.062550
http://dbpedia.org/ontology/weight,102.0,21384.0,0.004770,6337.0,103397.0,0.061288,448.0,6394.0,0.070066,102.0,6063.0,0.016823,1317.0,19869.0,0.066284,170.0,8882.0,0.019140,202.0,2303.0,0.087712,197.0,11475.0,0.017168,2.0,244.0,0.008197,61.0,1304.0,0.046779,1.0,13.0,0.076923,11,8939,181328,0.049297
http://dbpedia.org/ontology/postalCode,2010.0,151889.0,0.013233,2756.0,211078.0,0.013057,8560.0,111240.0,0.076951,21234.0,96986.0,0.218939,4650.0,22393.0,0.207654,80.0,115204.0,0.000694,5114.0,113158.0,0.045193,305.0,64946.0,0.004696,6462.0,161582.0,0.039992,7.0,54029.0,0.000130,31.0,68559.0,0.000452,11,51209,1171064,0.043729
http://dbpedia.org/ontology/populationTotal,280.0,18333.0,0.015273,7025.0,310209.0,0.022646,5883.0,174930.0,0.033631,59.0,97538.0,0.000605,357.0,219365.0,0.001627,19899.0,237147.0,0.083910,11975.0,83976.0,0.142600,3649.0,122839.0,0.029706,14703.0,161377.0,0.091110,112.0,171010.0,0.000655,13084.0,218776.0,0.059805,11,77026,1815500,0.042427
http://dbpedia.org/ontology/elevation,195.0,124489.0,0.001566,2063.0,222342.0,0.009278,14.0,914.0,0.015317,33.0,19697.0,0.001675,7002.0,24349.0,0.287568,9.0,4460.0,0.002018,563.0,27512.0,0.020464,112.0,26203.0,0.004274,2919.0,99453.0,0.029351,37.0,553.0,0.066908,1.0,1.0,1.000000,11,12948,549973,0.023543
http://dbpedia.org/ontology/areaCode,64.0,100097.0,0.000639,1110.0,134383.0,0.008260,5617.0,99317.0,0.056556,11.0,30741.0,0.000358,2782.0,30963.0,0.089849,12.0,51582.0,0.000233,35.0,21335.0,0.001640,3.0,20544.0,0.000146,1052.0,114514.0,0.009187,9.0,9126.0,0.000986,23.0,30856.0,0.000745,11,10718,643458,0.016657
http://dbpedia.org/ontology/activeYearsStartYear,186.0,47206.0,0.003940,3758.0,352908.0,0.010649,691.0,20500.0,0.033707,575.0,184387.0,0.003118,3.0,27366.0,0.000110,197.0,50530.0,0.003899,387.0,23084.0,0.016765,232.0,38767.0,0.005984,429.0,40361.0,0.010629,95.0,6703.0,0.014173,8.0,6529.0,0.001225,11,6561,798341,0.008218


## Export to LaTeX

In [56]:
cols_table3 = [f'{l}_pcnt' for l in langs]
cols_table3.extend(['num_langs', 'all_refs', 'all_total', 'all_pcnt'])
df_merged[cols_table3].sort_values(['num_langs', 'all_pcnt'], ascending=False).head(30)\
    .to_latex("tabular_properties_pcnt_referrenced.tex", float_format="%.3f", 
              formatters={'all_refs': "{:,}".format,'all_total': "{:,}".format})